In [ ]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
import glob
from shapely.geometry import Point, Polygon

In [ ]:
train = False
if train:
    images = glob.glob("./train/Task1/*.jpg")
else:
    images = glob.glob("./test/Task1/*.jpg")
images.sort()

In [ ]:
images_t = []
for img in images:
    images_t.append(cv.imread(img))

Hardcoded darts circle shapes (by numbers)

In [ ]:
h_s = {10: [(1296,1468),(1316,1484),(1288,1496),(1308,1508)], 9: [(1276,1352),(1348,1368),(1392,1412),(1404,1484),(1392,1557),(1364,1617),(1304,1637),(1236,1621),(1171,1545),(1175,1456),(1211,1396),(1236,1621)],
       8: [(1292,1219),(1368,1244),(1428,1288),(1456,1348),(1456,1352),(1484,1436),(1480,1528),(1456,1601),(1428,1681),(1368,1729),(1284,1761),(1163,1717),(1224,1753),(1099,1629),(1071,1512),(1087,1400),(1131,1308),(1203,1240)],
       7: [(1276,1107),(1364,1123),(1448,1179),(1500,1256),(1553,1356),(1569,1496),(1549,1605),(1512,1705),(1448,1801),(1364,1858),(1256,1874),(1135,1838),(1043,1749),(999,1633),(979,1500),(1007,1356),(1075,1215),(1167,1143)],
       6: [(1280,983),(1368,1003),(1464,1055),(1533,1115),(1585,1207),(1633,1312),(1649,1448),(1633,1585),(1605,1701),(1545,1809),(1472,1902),(1392,1954),(1292,2002),(1167,1986),(1067,1942),(979,1850),(906,1717),(878,1585),(874,1448),(902,1304),(971,1167),(1079,1055),(1187,999)],
       5: [(1272,878),(1175,886),(1071,931),(963,1011),(886,1103),(826,1219),(782,1340),(766,1476),(770,1605),(794,1725),(834,1825),(894,1926),(955,1998),(1047,2062),(1151,2110),(1264,2114),(1364,2078),(1460,2030),(1528,1970),(1601,1878),(1653,1789),(1380,890),(1488,951),(1565,1019),(1633,1115),(1677,1211),(1697,1308),(1713,1404),(1717,1520),(1697,1621),(1681,1709)],
       4: [(1224,738),(1340,758),(1448,798),(1541,862),(1613,935),(1681,1027),(1725,1127),(1761,1240),(1785,1356),(1789,1468),(1777,1589),(1753,1721),(1713,1821),(1653,1938),(1589,2034),(1500,2114),(1396,2183),(1288,2219),(1187,2227),(1063,2199),(951,2147),(850,2066),(766,1950),(706,1821),(670,1689),(654,1553),(666,1416),(698,1260),(742,1115),(814,995),(919,874),(1035,794),(1131,758)],
       3: [(1187,630),(1316,634),(1428,666),(1533,730),(1625,818),(1705,919),(1761,1027),(1809,1139),(1838,1260),(1850,1380),(1854,1512),(1833,1649),(1805,1777),(1769,1882),(1713,1982),(1657,2070),(1585,2155),(1504,2227),(1400,2283),(1280,2331),(1139,2331),(983,2303),(870,2239),(766,2143),(666,2014),(593,1858),(553,1693),(537,1553),(541,1392),(565,1232),(642,1047),(738,886),(850,766),(991,678),(1087,642)],
       2: [(1215,485),(1328,505),(1440,549),(1541,605),(1625,670),(1705,762),(1761,838),(1805,935),(1854,1031),(1886,1139),(1906,1244),(1918,1372),(1914,1500),(1906,1621),(1886,1737),(1854,1850),(1805,1954),(1753,2058),(1693,2147),(1617,2243),(1545,2311),(1444,2379),(1328,2431),(1224,2460),(1091,2464),(971,2439),(854,2383),(746,2315),(666,2235),(593,2138),(513,2010),(457,1862),(429,1729),(413,1565),(413,1412),(433,1256),(481,1087),(549,939),(642,786),(742,674),(882,577),(1027,505),(1143,489)],
       1: [(1183,333),(1324,353),(1452,393),(1557,457),(1653,537),(1737,626),(1801,722),(1874,842),(1922,967),(1966,1095),(1986,1219),(2002,1352),(1998,1484),(1986,1609),(1962,1737),(1934,1850),(1890,1970),(1846,2070),(1781,2167),(1717,2267),(1645,2347),(1557,2431),(1456,2492),(1344,2548),(1240,2580),(1115,2588),(975,2584),(850,2548),(742,2492),(650,2423),(545,2323),(473,2219),(393,2082),(333,1934),(292,1793),(268,1633),(268,1440),(276,1268),(309,1115),(357,971),(421,826),(513,690),(614,581),(742,469),(886,385),(1035,341)]}

Helper functions that identify the position on the dart board of a point by its (x, y) coordinates using the above hardcoded shapes

In [ ]:
def inside_zone(zone_no, point, h_s=h_s):
    if zone_no != 10:
        coord = h_s[zone_no]
        coord2 = h_s[zone_no+1]
    else:
        coord = h_s[10]

    p = Point(point[0], point[1])
    poly = Polygon(coord)

    if p.within(poly):
        if zone_no == 10:
            return True
        else:
            poly = Polygon(coord2)
            if not p.within(poly):
                return True
            else:
                return False
    else:
        return False

In [ ]:
def which_zone(point):
    zones = np.arange(1, 11)
    for zone in zones:
        if inside_zone(zone, point):
            return zone

In [ ]:
u = np.array([30, 255, 255])
l = np.array([8, 43, 56])

In [ ]:
masked = []

for img in images_t:
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    masked.append(cv.inRange(img_hsv, l, u))

In [ ]:
for i, img in enumerate(masked):
    line_coord = cv.HoughLinesP(img, 1, np.pi / 180, 40, minLineLength=51, maxLineGap=4)
    for coords in line_coord:
        #print(line)
        line = coords[0]
        cv.line(images_t[i], (line[0], line[1]), (line[2], line[3]), (255, 255, 0), 11)

In [ ]:
binary_highlighted = []

for img in images_t:
    binary_highlighted.append(cv.inRange(img, (255, 255, 0), (255, 255, 0)))       #(255, 255, 0) is the high and low hsv value

In [ ]:
binary_dilated = []
for img in binary_highlighted:
    binary_dilated.append(cv.dilate(img, np.ones((5, 5), np.uint8), iterations=1))

In [ ]:
positions_all = []                 #darts positions on darts board
nr = []                            #number of darts per image

In [ ]:
for img in binary_dilated:
    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    positions = []
    nr.append(len(contours))
    for contour in contours:
        M = cv.moments(contour)
        centroid = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        angle = 0.5 * np.arctan2(2 * M["mu11"], M["mu20"] - M["mu02"])
        x = -131 * np.cos(angle) + centroid[0]
        y = -131 * np.sin(angle) + centroid[1]
        pos = which_zone((x, y))
        positions.append(pos)
    positions_all.append(positions)

In [ ]:
try:
    os.mkdir("./Stegarescu_Ana-Maria_507")
except:
    print("Directory already exists")

Directory already exists


In [ ]:
try:
    os.mkdir("./Stegarescu_Ana-Maria_507/Task1")
except:
    print("Directory already exists")

Directory already exists


In [ ]:
for i in range(25):
    folder = 'Stegarescu_Ana-Maria_507/Task1'

    file = ''
    if i < 9:
        file = folder + '/0' + str(i+1) + '_predicted.txt'
    else:
        file = folder + "/" + str(i+1) + '_predicted.txt'
    f = open(file, "w")
    f.write(str(nr[i]) + "\n")
    for j, pos in enumerate(positions_all[i]):
        if j + 1 < len(positions_all[i]):
            f.write(str(pos) + "\n")
        else:
            f.write(str(pos))
    f.close()

In [ ]:
for i in range(25):
    print(i+1, positions_all[i])

1 [4, 7, 4]
2 [3, 6, 5]
3 [4, 6]
4 [5, 4]
5 [6, 5]
6 [5]
7 [7, 9, 6]
8 [6, 7]
9 [5, 4]
10 [7, 9]
11 [8]
12 [6, 6, 7]
13 [9, 6, 6]
14 [6]
15 [4, 7]
16 [6, 7]
17 [9]
18 [8, 4]
19 [6]
20 [6, 4]
21 [4, 7, 4]
22 [6, 8, 4]
23 [5]
24 [5, 4]
25 [4]
